In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option('display.width', 100)

import hvplot.pandas

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")

In [3]:
# Fill NANs with 0 value and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita']=np.where(df['population']!=0, df['gdp']/df['population'],0)
df

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,oil_co2,oil_co2_per_capita,other_industry_co2,other_co2_per_capita,co2_growth_prct,co2_growth_abs,co2_per_gdp,co2_per_unit_energy,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,...,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,share_global_cumulative_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,total_ghg,ghg_per_capita,total_ghg_excluding_lucf,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp,gdp_per_capita
0,AFG,Afghanistan,1949,0.015,0.002,0.000,0.000,0.000,0.015,0.002,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,7624058.0,0.000000e+00,0.000,0.000,0.000,0.000000
1,AFG,Afghanistan,1950,0.084,0.011,0.000,0.000,0.000,0.021,0.003,0.0,0.0,0.0,0.0,0.063,0.008,0.0,0.0,475.00,0.070,0.009,0.000,0.000,0.000,0.000,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,7752117.0,9.421400e+09,0.000,0.000,0.000,1215.332543
2,AFG,Afghanistan,1951,0.092,0.012,0.000,0.000,0.000,0.026,0.003,0.0,0.0,0.0,0.0,0.066,0.008,0.0,0.0,8.70,0.007,0.010,0.000,0.000,0.000,0.000,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,7840151.0,9.692280e+09,0.000,0.000,0.000,1236.236369
3,AFG,Afghanistan,1952,0.092,0.012,0.000,0.000,0.000,0.032,0.004,0.0,0.0,0.0,0.0,0.060,0.008,0.0,0.0,0.00,0.000,0.009,0.000,0.000,0.000,0.000,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,7935996.0,1.001733e+10,0.000,0.000,0.000,1262.264378
4,AFG,Afghanistan,1953,0.106,0.013,0.000,0.000,0.000,0.038,0.005,0.0,0.0,0.0,0.0,0.068,0.008,0.0,0.0,16.00,0.015,0.010,0.000,0.000,0.000,0.000,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,8039684.0,1.063052e+10,0.000,0.000,0.000,1322.255925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25984,ZWE,Zimbabwe,2016,10.738,0.765,1.415,0.639,0.046,6.959,0.496,0.0,0.0,0.0,0.0,3.139,0.224,0.0,0.0,-12.17,-1.488,0.512,0.230,12.153,0.866,0.580,...,0.0,0.0,0.03,0.0,0.05,0.05,0.08,0.0,0.0,0.02,0.0,116.52,8.305,29.12,2.076,11.50,0.820,6.21,0.443,14030338.0,2.096179e+10,46.666,3326.073,2.226,1494.033433
25985,ZWE,Zimbabwe,2017,9.582,0.673,1.666,0.678,0.048,5.665,0.398,0.0,0.0,0.0,0.0,3.239,0.228,0.0,0.0,-10.77,-1.156,0.437,0.209,11.248,0.790,0.512,...,0.0,0.0,0.03,0.0,0.05,0.05,0.08,0.0,0.0,0.02,0.0,116.09,8.154,28.80,2.023,11.62,0.816,6.35,0.446,14236599.0,2.194784e+10,45.936,3226.617,2.093,1541.648846
25986,ZWE,Zimbabwe,2018,11.854,0.821,1.308,0.697,0.048,7.101,0.492,0.0,0.0,0.0,0.0,4.056,0.281,0.0,0.0,23.72,2.273,0.522,0.250,13.163,0.912,0.580,...,0.0,0.0,0.03,0.0,0.05,0.05,0.08,0.0,0.0,0.02,0.0,118.77,8.226,31.38,2.173,11.96,0.828,6.59,0.456,14438812.0,2.271535e+10,47.502,3289.887,2.091,1573.214825
25987,ZWE,Zimbabwe,2019,10.949,0.748,1.473,0.697,0.048,6.020,0.411,0.0,0.0,0.0,0.0,4.232,0.289,0.0,0.0,-7.64,-0.905,0.000,0.222,12.422,0.848,0.000,...,0.0,0.0,0.03,0.0,0.05,0.05,0.08,0.0,0.0,0.02,0.0,0.00,0.000,0.00,0.000,0.00,0.000,0.00,0.000,14645473.0,0.000000e+00,49.427,3374.877,0.000,0.000000


In [4]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## 1) CO2 Emission over time by continent

In [5]:
#Define Panel Widgets
year_slider = pn.widgets.IntSlider(name='Year Slider', start=1750, end=2020, step=5, value=1850)

In [6]:
#Create Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(name='Y axis', options=['co2', 'co2_per_capita',], button_type='success')

In [7]:
#linking dataset to interactive widgets
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (idf[(idf.year<=year_slider) & (idf.country.isin(continents))]
                .groupby(['country', 'year'])[yaxis_co2].mean()
                .to_frame()
                .reset_index()
                .sort_values(by=['year'])
                .reset_index(drop = True)
               )

In [8]:
co2_pipeline

In [9]:
co2_plot = co2_pipeline.hvplot(x='year', by='country', y=yaxis_co2, line_width=2, title="CO2 emission by continent")
co2_plot

## 2) Table - CO2 emission over time by continent

In [10]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')
co2_table

## 3) CO2 vs GDP scatterplot

In [11]:
co2_vs_gdp_scatterplot_pipeline = (idf[(idf.year == year_slider) & (~ (idf.country.isin(continents)))].groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
                                   .to_frame()
                                   .reset_index()
                                   .sort_values(by=['year'])
                                   .reset_index(drop=True)
                                  )

In [12]:
co2_vs_gdp_scatterplot_pipeline

In [13]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', y='co2', by='country', size=80, kind='scatter', title="CO2 emission vs GDP per Capita", legend=True, height=600, width=1000)
co2_vs_gdp_scatterplot                                                             

## 4) Bar Chart with CO2 sources by continent

In [14]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['coal_co2', 'oil_co2', 'gas_co2'],
    button_type='success',
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
co2_source_bar_pipeline = (
    idf[(idf.year==year_slider) & (idf.country.isin(continents_excl_world))]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=['year'])
    .reset_index(drop=True)
)

In [15]:
co2_source_bar_pipeline

In [16]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', x='country', y=yaxis_co2_source, title = 'CO2 source by continent')
co2_source_bar_plot                                                  

## Creating Dashboard

In [26]:
#layout using Panel template
#Dashboard one side bar and beside the sidebar, there are two rows; each has two columns.
template = pn.template.FastListTemplate(
    title='World CO2 Emission Dashboard',
    sidebar=[pn.pane.Markdown('#CO2 Emissions and Climate Change'),
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It's widely recognised that to avoid Global warming, carbon dioxide emission has to be reduced."),
             pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown('## Settings'),
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=700), margin=(0,25)), pn.Column(co2_table.panel(width=500))), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=1000), margin=(0,25)), pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color = "#88d8b0",
    header_background = "#88d8b0",
)
#Template.show()    
template.servable();